In [ ]:
from itertools import product
from pathlib import Path
import astropy
from astropy.table import QTable
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import scipy as sp
from astropy.visualization import quantity_support
quantity_support()
from tqdm import tqdm
import astropy.units as u
from scipy.interpolate import interp1d
from astropy.units import Quantity
from joblib import Parallel, delayed
import os
from astropy.io import fits
from astropy.units import Unit

from spots_and_faculae_model.spectrum import spectrum
from spots_and_faculae_model.simpler_spectral_grid import simpler_spectral_grid

HARPS_WAVELENGTH_UNITS : Unit = u.Angstrom
HARPS_FLUX_UNITS : Unit = u.MJy

def read_HARPS_fits(fits_absolute_path : Path, verbose : bool = False, name : str = None, INTEGRATION_INDEX : int = 0) -> spectrum:
	"""
	Attributes
	----------
	verbose : bool (default False)
		prints a summary of all the headers found in the fits file, as well as the string representation of the header with header index HDU_INDEX
	"""
	if not fits_absolute_path.exists():
		raise FileNotFoundError(f"JWST spectrum file not found at : {fits_absolute_path}.")

	with fits.open(fits_absolute_path) as hdul:
		HDU_INDEX = 1 # aka EXTRACT1D
		

		data = hdul[HDU_INDEX].data
		hdr = hdul[HDU_INDEX].header
		
		if verbose:
			hdul.info()
			print(repr(hdr))

		# these column name strings are unique to JWST 1D 
		spec : spectrum = spectrum(wavelengths = data["WAVE"][INTEGRATION_INDEX] * HARPS_WAVELENGTH_UNITS,
				  fluxes = data["FLUX"][INTEGRATION_INDEX] * HARPS_FLUX_UNITS, name=name, normalised_point=5000*u.Angstrom, normalise_flux=True)
		
	return spec

			


In [ ]:
# for now, just request a random composite spectrum from facula_and_spot_creator
# and try to decompose it - aka can we regenerate the w's

# eventually can read in external data or some training data from a large hdf5 file etc

from pathlib import Path
import numpy as np
from astropy.visualization import quantity_support
quantity_support()
import astropy.units as u
import os

from spots_and_faculae_model.spectrum import spectrum
from spots_and_faculae_model.simpler_spectral_grid import simpler_spectral_grid

external_spectrum_path = Path("../../assets/ADP.2016-02-04T01_02_52.843.fits")
script_dir = os.getcwd()  # usually the folder where notebook is running")
wavelength_grid_absolute_path = (script_dir / external_spectrum_path).resolve()

spectrum_to_decompose : spectrum = read_HARPS_fits(wavelength_grid_absolute_path, INTEGRATION_INDEX=0, verbose=True)
spectrum_to_decompose.plot()
print(spectrum_to_decompose.Wavelengths)

mask = np.isfinite(spectrum_to_decompose.Fluxes)

spectrum_to_decompose = spectrum_to_decompose[mask]

print("reading in hdf5")
spectral_grid_relative_path = Path("../../assets/JWST_convolved_spectral_grid.hdf5")
spectral_grid_absolute_path = (script_dir / spectral_grid_relative_path).resolve()
spec_grid : simpler_spectral_grid = simpler_spectral_grid.from_hdf5(absolute_path=spectral_grid_absolute_path)
lookup_table = spec_grid.to_lookup_table()
print("finished reading in hdf5")

print(lookup_table[2500 * u.K, 0.0 * u.dimensionless_unscaled, 0.0 * u.dimensionless_unscaled])